In [43]:
import gurobipy as gp

#Abrindo a intancia
def build_instance(file_name):
    print(f"BUILDING INSTANCE {file_name.split("/")[len(file_name.split("/"))-1]}")
    instance_file = open(file_name,"r")
    
    lines = [line.strip() for line in instance_file.readlines()]
    current_line = 0
    
    #Salvando quantidade de clientes e quantidade e lockers
    qty_customer = int(lines[current_line].split(" ")[0])
    qty_locker = int(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    max_vehicle = int(lines[current_line].split(" ")[0])
    vehicle_capacity = float(lines[current_line].split(" ")[1])
    
    current_line += 1
    
    demands = list()
    ## Salvando demands de clients
    for demand in range(current_line,qty_customer+2):
        demands.append(float(lines[demand]))
    
    current_line += qty_customer
    
    #Preenchendo nos
    nodes = dict()
    
    #Deposito
    depot = {
        'label': "D",
        'x' : float(lines[current_line].split(" ")[0]),
        'y' : float(lines[current_line].split(" ")[1]),
        'earliest' : float(lines[current_line].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[current_line].split(" ")[4]),
        'demand': 0,
        'type' : int(lines[current_line].split(" ")[5])
    }
    nodes['D']=depot
    current_line += 1

    #Clientes
    customers = dict()
    customer_count = 1
    for i in range(current_line, current_line + qty_customer):
        customer = {
        'label' : f"C{customer_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': demands[customer_count-1],
        'type' : int(lines[i].split(" ")[5])
        }
        customer_count+=1
        customers[customer['label']]=customer
        nodes[customer['label']]=customer
    current_line += qty_customer

    #Lockers
    lockers = dict()
    locker_count = 1
    for i in range(current_line,current_line+qty_locker):
        locker = {
        'label' : f"P{locker_count}",
        'x' : float(lines[i].split(" ")[0]),
        'y' : float(lines[i].split(" ")[1]),
        'earliest' : float(lines[i].split(" ")[2]),
        'latest' : float(lines[current_line].split(" ")[3]),
        'service_time' : float(lines[i].split(" ")[4]),
        'demand': 0,
        'customers': list(),
        'type' : int(lines[i].split(" ")[5])
        }
        locker_count+=1
        lockers[locker['label']] = locker
        nodes[locker['label']]=locker
    current_line += qty_locker

    #Atribuição de lockers
    current_customer = 0
    for i in range(current_line, current_line+qty_customer):
        try:
            customer = customers[list(customers.keys())[current_customer]]
            
            locker_index = lines[i].split(" ").index("1")
            locker = lockers[list(lockers.keys())[locker_index]]
            customer['locker'] = locker['label']
            locker['customers'].append(customer['label'])
        except ValueError:    
            customer['locker'] = ""
        current_customer += 1
    current_line += qty_customer
    
    #Matriz de Distancia
    costs = dict()
    for i,node_i in enumerate(nodes):
         current_node_i = nodes[node_i]
         current_node_i['index'] = i
         for j,node_j in enumerate(nodes):
             current_node_j = nodes[node_j]
             distance = ((current_node_i['x']-current_node_j['x'])**2 + (current_node_i['y']-current_node_j['y'])**2)**(1/2)
             costs[current_node_i['label'],current_node_j['label']] = distance
    print(f"Qty customer:{qty_customer}")
    print(f"Qty locker:{qty_locker}")
    print(f"Max vehicles:{max_vehicle}")
    print(f"Vehicle capacity:{vehicle_capacity}")
    print(f"BUILD FINISHED")
    print(sum(demands))
    return {
        'qty_customer': qty_customer,
        'qty_locker': qty_locker,
        'qty_nodes': qty_locker,
        'total_demands': sum(demands),
        'min_vehicles': round(sum(demands)/vehicle_capacity),
        'max_vehicle': max_vehicle,
        'vehicle_capacity': vehicle_capacity,
        'nodes': nodes,
        'customers': customers,
        'lockers': lockers,
        'depot': depot,
        'costs': costs
    }


instance = build_instance("../../instances/vrppl/inst_test")

BUILDING INSTANCE inst_test
Qty customer:10
Qty locker:2
Max vehicles:2
Vehicle capacity:200.0
BUILD FINISHED
150.0


In [44]:
qty_vehicles = 2

#Rótulos

nodes = instance['nodes']
node_labels = list(nodes.keys())
locker_labels = [node for node in node_labels if nodes[node]['type'] == 4]
customer_labels = list(instance['customers'].keys())
customers_hc = [customer for customer in customer_labels if nodes[customer]['type'] == 1]
customers_lc = [customer for customer in customer_labels if nodes[customer]['type'] == 2]
customers_hlc = [customer for customer in customer_labels if nodes[customer]['type'] == 3]

#Veiculos
vehicles = list()
for i in range(qty_vehicles):
    vehicles.append(f"v_{i}")


In [45]:
#Criando dicts

#Custos
costs = instance['costs']

In [46]:
#Criando modelo
m = gp.Model()
m.setParam(gp.GRB.Param.OutputFlag,0)

#Inserindo variaveis
x_ijlk = m.addVars(node_labels,node_labels,customer_labels,vehicles, vtype = gp.GRB.BINARY)

# print(x_ijlk)

In [53]:
#Definindo restrições

# Cliente home só pode ser atendido se o no de destino for ele mesmo
c1 = m.addConstrs(
    x_ijlk[node_i,node_j,customer_l,vehicle] == 0
    for node_i in node_labels
    for node_j in node_labels
    for customer_l in customers_hc
    for vehicle in vehicles
    if node_j != customer_l
)
#Os clientes home deve ser atendidos em casa
c2 = m.addConstrs(
    gp.quicksum(x_ijlk[node_i,node_j,customer_l, vehicle] for node_i in node_labels for node_j in node_labels for vehicle in vehicles) == 1
    for customer_l in customers_hc
)

# Cliente locker só pode ser atendido se o no de destino for o locker dele
# c1 = m.addConstrs(
#     x_ijlk[node_i,node_j,customer_l,vehicle] == 0
#     for node_i in node_labels
#     for node_j in node_labels
#     for customer_l in customers_lc
#     for vehicle in vehicles
#     if node_j != nodes[customer_l]['locker'] and nodes[node_j][
# )

c4 = m.addConstrs(
    gp.quicksum(x_ijlk[node_i,node_j,customer_l, vehicle] for node_i in node_labels for node_j in node_labels for vehicle in vehicles) == 1
    for customer_l in customers_hc
)



#Definindo função objetivo
m.setObjective(
    gp.quicksum(costs[node_i,node_j] * x_ijlk[node_i,node_j,customer_l,vehicle] for node_i in node_labels for node_j in node_labels for customer_l in customer_labels for vehicle in vehicles),
    sense = gp.GRB.MINIMIZE
)


#Executando modelo

m.optimize()
print(f"Valor Total: {m.objVal}\n")

for vehicle in vehicles:
    print(vehicle)
    for node_i in node_labels:
        for node_j in node_labels:
            for customer_l in customer_labels:
                if round(x_ijlk[node_i,node_j,customer_l,vehicle].X) == 1:
                    if node_j in locker_labels:
                        print(f"{node_i},{node_j}({customer_l})", " ", end="")
                    else:
                        print(f"{node_i},{node_j}", " ", end="")
    print('\n')

Valor Total: 0.0

v_0
C9,C9  C10,C10  

v_1
C7,C7  

